# CropSense AI: Crop Disease Detection in Rwanda

Objective: This notebook explores classification models for plant disease detection using classical ML and neural networks, applying optimization techniques to improve performance.

A Notebook detailing the following

* Project name





**Instructions**

1. Acquire a dataset suitable for ML tasks as per your proposal.
2. Implement a simple machine learning model based on neural networks on the chosen dataset without any defined optimization techniques. (Check instructions)
3. Implement and compare the model's performance after applying 3 to 4 disntict combinations regularization and optimization techniques.
4. Discuss the results on the README file.
5. Make predictions using test data
7. Implement error analysis techniques and ensure there is: F1-Score, Recall, Precision, RUC a confusion matrix using plotting libraries (not verbose)

Submit notebook to github repo



Step 1: Setup and Install Dependencies

In [ ]:
!pip install tensorflow tensorflow-datasets seaborn joblib


Step 2: Imports & Global Setup

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from joblib import dump

# SECTION 1: Model Architecture

In [ ]:
from IPython.display import Image, display
display(Image("model_architecture.png", width=400))

Step 3: Load PlantVillage Dataset



In [ ]:
# Load PlantVillage from TFDS
(ds_train, ds_val), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Resize, normalize, and batch
IMG_SIZE = 64
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(1)

# Check class names
class_names = ds_info.features['label'].names
print("Detected classes:", class_names)
NUM_CLASSES = len(class_names)

Step 4: Define define_model() Function

In [ ]:
def define_model(optimizer='adam', regularizer=None, early_stopping=False, dropout_rate=0.3, lr=0.001, layers_num=2):
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)))

    for _ in range(layers_num):
        model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizer))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

    opt = tf.keras.optimizers.get(optimizer)
    opt.learning_rate = lr

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    callbacks = []
    if early_stopping:
        callbacks.append(EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True))

    return model, callbacks

Instance 1: Baseline Model (No Optimization)

In [ ]:
# Instance 1: Baseline model (no optimizer tuning, no early stopping, default settings)

model_1, callbacks_1 = define_model(
    optimizer='adam',
    regularizer=None,
    early_stopping=False,
    dropout_rate=0.0,
    lr=0.001,
    layers_num=2
)

history_1 = model_1.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=callbacks_1,
    verbose=1
)

# Save model
os.makedirs("saved_models", exist_ok=True)
model_1.save("saved_models/model_instance_1.keras")


Evaluate & Visualize

In [ ]:
# Plot loss curve
plt.plot(history_1.history['loss'], label='Train Loss')
plt.plot(history_1.history['val_loss'], label='Val Loss')
plt.title("Instance 1: Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Evaluate model on validation set
y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model_1.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Instance 1 Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, cmap='Blues')
plt.title("Instance 1: Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Instance 2: RMSprop + L2 Regularization + Dropout + Early Stopping

In [ ]:
l2_reg = regularizers.l2(0.01)

model_2, callbacks_2 = define_model(
    optimizer='rmsprop',
    regularizer=l2_reg,
    early_stopping=True,
    dropout_rate=0.3,
    lr=0.0005,
    layers_num=3
)

history_2 = model_2.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=callbacks_2,
    verbose=1
)

# Save model
model_2.save("saved_models/model_instance_2.keras")

Evaluation Cell

In [ ]:
# Plot loss curve
plt.plot(history_2.history['loss'], label='Train Loss')
plt.plot(history_2.history['val_loss'], label='Val Loss')
plt.title("Instance 2: Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Evaluation
y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model_2.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Instance 2 Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, cmap='Purples')
plt.title("Instance 2: Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Instance 3: Adam + L1 Regularization + Dropout + More Layers

In [ ]:
l1_reg = regularizers.l1(0.01)

model_3, callbacks_3 = define_model(
    optimizer='adam',
    regularizer=l1_reg,
    early_stopping=True,
    dropout_rate=0.4,
    lr=0.0003,
    layers_num=4
)

history_3 = model_3.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=callbacks_3,
    verbose=1
)

model_3.save("saved_models/model_instance_3.keras")

 Evaluation Cell

In [ ]:
# Plot loss curve
plt.plot(history_3.history['loss'], label='Train Loss')
plt.plot(history_3.history['val_loss'], label='Val Loss')
plt.title("Instance 3: Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Evaluation
y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model_3.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Instance 3 Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, cmap='Reds')
plt.title("Instance 3: Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Instance 4: RMSprop + L1_L2 Combo + NO EarlyStopping

In [ ]:
l1_l2_reg = regularizers.l1_l2(l1=0.005, l2=0.005)

model_4, callbacks_4 = define_model(
    optimizer='rmsprop',
    regularizer=l1_l2_reg,
    early_stopping=False,
    dropout_rate=0.2,
    lr=0.0007,
    layers_num=3
)

history_4 = model_4.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=callbacks_4,
    verbose=1
)

model_4.save("saved_models/model_instance_4.keras")

Evaluation Cell

In [ ]:
# Plot loss curve
plt.plot(history_4.history['loss'], label='Train Loss')
plt.plot(history_4.history['val_loss'], label='Val Loss')
plt.title("Instance 4: Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Evaluation
y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model_4.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Instance 4 Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, cmap='Greens')
plt.title("Instance 4: Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Instance 5: Adam + No Regularization + High Dropout + Tiny LR


In [ ]:
model_5, callbacks_5 = define_model(
    optimizer='adam',
    regularizer=None,
    early_stopping=True,
    dropout_rate=0.5,
    lr=0.0001,
    layers_num=3
)

history_5 = model_5.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=callbacks_5,
    verbose=1
)

model_5.save("saved_models/model_instance_5.keras")

Evaluation Cell

In [ ]:
# Plot loss curve
plt.plot(history_5.history['loss'], label='Train Loss')
plt.plot(history_5.history['val_loss'], label='Val Loss')
plt.title("Instance 5: Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Evaluation
y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model_5.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Instance 5 Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, cmap='Oranges')
plt.title("Instance 5: Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Best Model

In [ ]:
from tensorflow.keras.models import load_model
import shutil

# Load all saved models and evaluate
model_paths = {
    "model_instance_1": "saved_models/model_instance_1.keras",
    "model_instance_2": "saved_models/model_instance_2.keras",
    "model_instance_3": "saved_models/model_instance_3.keras",
    "model_instance_4": "saved_models/model_instance_4.keras",
    "model_instance_5": "saved_models/model_instance_5.keras"
}

best_accuracy = 0
best_model_name = ""
best_model_path = ""

for name, path in model_paths.items():
    model = load_model(path)
    y_true, y_pred = [], []
    for images, labels in ds_val:
        preds = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    acc = accuracy_score(y_true, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")

    if acc > best_accuracy:
        best_accuracy = acc
        best_model_name = name
        best_model_path = path

# Save best model as 'best_model.keras'
shutil.copy(best_model_path, "saved_models/best_model.keras")
print(f"\n Best model is {best_model_name} with accuracy = {best_accuracy:.4f}")